In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import sys
sys.path.append('../pipeline/src')

import torch
import pandas as pd
import numpy as np
from datamodules.paired_dataset import PairedDataset
from datamodules.paired_datamodule import PairedProteinDataModule, PairedProteinDataModuleConfig
from datamodules.datamodule import ProteinDataModuleConfig, SamplingConfig, ProteinDataModule
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from pprint import pprint









In [22]:
# Create paired dataset from DataFrames
import pandas as pd

train_dataset_path = '../pipeline/output/datasets/train.pt'
val_dataset_path = '../pipeline/output/datasets/val.pt'
test_dataset_path = '../pipeline/output/datasets/test.pt'
similarity_file_path = '../pipeline/output/mmseqs/all_seqs_similarity.tsv'
cluster_path = '../pipeline/output/mmseqs/all_seqs_clust.tsv'


In [23]:

sampling_conf = SamplingConfig(
    cluster_path=cluster_path,
    train_n_pdb=1.0,
    train_intersect_val_clusters=True,
    train_intersect_test_clusters=True,
    val_n_pdb=1.0,
    test_n_pdb=1.0,
    train_max_chains_per_pdb=1,
    val_max_chains_per_pdb=1,
    test_max_chains_per_pdb=1,
)
paired_datamodule_config = PairedProteinDataModuleConfig(
    _target_="pipeline.src.datamodules.paired_datamodule.PairedProteinDataModuleConfig",
    name="paired_datamodule",
    train_dataset_path=train_dataset_path,
    val_dataset_path=val_dataset_path,
    test_dataset_path=test_dataset_path,
    tokenizer_name="facebook/esm2_t33_650M_UR50D",
    batch_size=1,
    num_workers=1,
    max_seq_length=1024,
    contact_threshold=8.0,
    sampler=sampling_conf,
    similarity_file_path=similarity_file_path,
    min_similarity_threshold=0.3,
)

paired_datamodule = PairedProteinDataModule(paired_datamodule_config)
paired_datamodule.setup()




INFO:datamodules.paired_datamodule:Num train in queries: 16015
INFO:datamodules.paired_datamodule:Num test in queries: 586
INFO:datamodules.paired_datamodule:Loaded 16601 similarity pairs: 
INFO:datamodules.paired_datamodule:                 target_id  identity
query_id                            
train_4MYS_A  train_1R3D_A     0.387
train_4F1K_A  train_4HQF_A     0.928
train_6PK0_C  train_6W5G_B     0.426
train_6K6W_C  train_7WNT_A     0.389
train_3RQ0_A  train_4W65_A     0.924
train_150L_A  train_3HUA_A     0.949
train_8IQB_A  train_8IQD_B     0.927
train_7CI5_A  train_3UHM_A     0.940
train_2O5M_X  train_1A6K_A     0.884
train_2QB5_B  train_2Q7D_B     0.931
INFO:datamodules.datamodule:Sample Train: After selecting 1.0 n/frac of pdbs, train dataset size: 24245


             id pdb_id chain_id  length  \
0  train_106M_A   106M        A     154   
1  train_109L_A   109L        A     162   
2  train_111M_A   111M        A     154   
3  train_115L_A   115L        A     162   
4  train_122L_A   122L        A     162   

                                     residue_indices  mean_b_factor  
0  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...      16.246299  
1  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      14.398765  
2  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...      20.670714  
3  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      13.986358  
4  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      14.248951  
             id pdb_id chain_id  length  \
0  train_1A77_A   1A77        A     315   
1  train_1AQJ_A   1AQJ        A     381   
2  train_1AQJ_B   1AQJ        B     383   
3  train_1C1K_A   1C1K        A     217   
4  train_1C90_A   1C90        A     265   

                                     residue_indices  mean_b_f

/workspace/esm-2-contact-map/ipynb/../pipeline/src/datamodules/datamodule.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pdb = df.groupby("pdb_id").apply(
INFO:datamodules.datamodule:Sample Train: After selecting 1 chains per pdb, train dataset size: 14503
INFO:datamodules.datamodule:Sample Train: After sampling, train dataset size: 14503
INFO:datamodules.datamodule:Sample Val: After selecting 1.0 n/frac of pdbs, val dataset size: 767
/workspace/esm-2-contact-map/ipynb/../pipeline/src/datamodules/datamodule.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the

In [24]:
for i, batch in tqdm(enumerate(paired_datamodule.train_dataloader())):
    print(f"OUR BATCH {i}")
    # print(batch)
    if i>10:
        break

11it [00:00, 130.07it/s]


OUR BATCH 0
OUR BATCH 1
OUR BATCH 2
OUR BATCH 3
OUR BATCH 4
OUR BATCH 5
OUR BATCH 6
OUR BATCH 7
OUR BATCH 8
OUR BATCH 9
OUR BATCH 10
OUR BATCH 11


# init model

In [25]:
from hydra import compose, initialize
from omegaconf import OmegaConf


with initialize(version_base=None, config_path="../pipeline/conf"):
    cfg = compose(config_name="config", overrides=["model=esm2_lora_template_contactconv_650m"])

In [26]:
from models.esm2_lora_template_contactconvhead import ESM2LoRAContactConvTemplate, ESM2LoRAContactConvTemplateConfig












In [27]:
conf = ESM2LoRAContactConvTemplateConfig(**cfg.model)

In [28]:
pprint(conf)

ESM2LoRAContactConvTemplateConfig(_target_='esm2_lora_contactconv_template',
                                  name='esm2 lora with conv refiner and '
                                       'template',
                                  backbone='facebook/esm2_t33_650M_UR50D',
                                  loss=LossConfig(loss_type='bce',
                                                  focal_gamma=2.0,
                                                  focal_alpha=-1),
                                  wo_lora=False,
                                  lora_rank=8,
                                  lora_alpha=16,
                                  lora_dropout=0.1,
                                  target_modules=['query', 'key', 'value'],
                                  contact_head_dim=4,
                                  learning_rate=0.0005,
                                  weight_decay=0.0001,
                                  backbone_init_from='../data/checkpoints/best_lora_

In [93]:
model = ESM2LoRAContactConvTemplate(conf)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:models.esm2_lora_contacthead:Loading backbone from ../data/checkpoints/best_lora_conv_refiner_k7.ckpt


In [60]:
from itertools import islice

batches10 = islice(paired_datamodule.train_dataloader(),10)

In [87]:
# model.validation_step_outputs.clear()
for batch in batches10:
    model.validation_step(batch,1)
# model.on_validation_epoch_end()

In [42]:
batch['similar_sequence']['input_ids'].shape

torch.Size([1, 229])

In [43]:
batch['similar_sequence']['contact_maps'].shape

torch.Size([1, 227, 227])

In [46]:
batch['primary_sequence']['contact_maps'].shape

torch.Size([1, 226, 226])

In [90]:
model.freeze_lora()

In [95]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

esm_model.base_model.model.embeddings.word_embeddings.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.query.base_layer.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.query.base_layer.bias False
esm_model.base_model.model.encoder.layer.0.attention.self.query.lora_A.default.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.query.lora_B.default.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.key.base_layer.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.key.base_layer.bias False
esm_model.base_model.model.encoder.layer.0.attention.self.key.lora_A.default.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.key.lora_B.default.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.value.base_layer.weight False
esm_model.base_model.model.encoder.layer.0.attention.self.value.base_layer.bias False
esm_model.base_model.model.encoder.layer.0.attentio